In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0240f3d18f736055343bf569dbda16f14d6c02131f8baa91e59bfd5dde23dd9c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
os.environ['JAVA_HOME'] = 'D:\\Java\\jdk1.8.0_202\\'

In [7]:
from pyspark.sql import SQLContext

In [8]:
spark = SparkSession \
    .builder \
    .appName("ML_Regression_Example2") \
    .getOrCreate()

# Create Spark Context

In [10]:
sc = spark.sparkContext


sqlContext = SQLContext(sc)

d:\Python311\Lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
from google.colab import files
uploaded = files.upload()


Saving daily_weather.csv to daily_weather.csv


In [5]:
file='daily_weather.csv'

df = sqlContext.read.load(file,
                          format='com.databricks.spark.csv',
                          header='true',inferSchema='true')

In [6]:
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [7]:
df.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [9]:
df.describe().toPandas().transpose()

0                    1                   2  \
summary                 count                 mean              stddev   
number                   1095                547.0  316.24357700987383   
air_pressure_9am         1092    918.8825513138094   3.184161180386833   
air_temp_9am             1090    64.93300141287072  11.175514003175877   
avg_wind_direction_9am   1091    142.2355107005759   69.13785928889189   
avg_wind_speed_9am       1092     5.50828424225493  4.5528134655317185   
max_wind_direction_9am   1092   148.95351796516923   67.23801294602953   
max_wind_speed_9am       1091    7.019513529175272   5.598209170780958   
rain_accumulation_9am    1089  0.20307895225211126  1.5939521253574893   
rain_duration_9am        1092    294.1080522756142  1598.0787786601481   
relative_humidity_9am    1095    34.24140205923536  25.472066802250055   
relative_humidity_3pm    1095    35.34472714825898  22.524079453587273   

                                         3                   4  
summary                                min                 max  
number                                   0                1094  
air_pressure_9am         907.9900000000024   929.3200000000012  
air_temp_9am            36.752000000000685   98.90599999999992  
avg_wind_direction_9am  15.500000000000046               343.4  
avg_wind_speed_9am        0.69345139999974  23.554978199999763  
max_wind_direction_9am   28.89999999999991  312.19999999999993  
max_wind_speed_9am      1.1855782000000479   29.84077959999996  
rain_accumulation_9am                  0.0   24.01999999999907  
rain_duration_9am                      0.0             17704.0  
relative_humidity_9am    6.090000000001012    92.6200000000002  
relative_humidity_3pm   5.3000000000006855    92.2500000000003

In [10]:
df.describe('air_pressure_9am').show()

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1092|
|   mean|918.8825513138094|
| stddev|3.184161180386833|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+



In [11]:
len(df.columns)

11

In [12]:
df.count()

1095

In [13]:
df2 = df.na.drop(subset=['air_pressure_9am'])

In [14]:
df2.count()

1092

In [15]:
df2.stat.corr("rain_accumulation_9am", "rain_duration_9am")

0.7298253479609021

#  Regression

In [16]:
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml.feature import VectorAssembler

In [17]:
df.count(), len(df.columns)

(1095, 11)

In [18]:
df = df.drop('number')

In [19]:
df = df.na.drop()

In [20]:
df.count(), len(df.columns)

(1064, 10)

In [21]:
df.columns

['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [22]:
featureColumns =df.columns[:-2]

In [32]:
featureColumns

['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am']

In [24]:
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
assembled = assembler.transform(df)

In [25]:
assembled.show(10)

+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+--------------------+
| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|            features|
+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+--------------------+
|918.0600000000087| 74.82200000000041|                 271.1| 2.080354199999768|    295.39999999999986| 2.863283199999908|                  0.0|              0.0|    42.42000000000046|   36.160000000000494|[918.060000000008...|
|917.3476881177097| 71.40384263106537|    101.93517935618371|2.4430092157340217|    140.

In [33]:
select_assembled=assembled.select("relative_humidity_3pm","features")
select_assembled=select_assembled.withColumnRenamed("relative_humidity_3pm","label")

In [34]:
(trainData, testData) = select_assembled.randomSplit([0.8,0.2], seed = 13234 )

In [35]:
trainData.count(),testData.count()

(846, 218)

In [36]:
from pyspark.ml.regression import LinearRegression


In [37]:
trainData.show()

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
|5.3000000000006855|[921.170000000008...|
| 5.360000000001202|[919.180000000011...|
| 6.467358677145455|[922.221203584900...|
| 6.520000000000103|[922.69000000001,...|
| 7.187360984546972|[922.928408958627...|
| 7.220000000001187|[920.400000000010...|
| 7.302180510020005|[922.273264482375...|
| 7.362581098053454|[920.632354382020...|
|7.4200000000011475|[919.500000000011...|
|   7.5200000000001|[920.410000000007...|
| 7.740000000000088|[922.040000000009...|
| 7.910000000000353|[921.220000000001...|
| 7.946408490460457|[920.300167510041...|
|  8.03000000000041|[921.720000000002...|
| 8.030000000001184|[919.700000000012...|
| 8.166229190843868|[920.791124402495...|
|  8.29967406061132|[920.567764807644...|
| 8.560000000001134|[917.440000000011...|
| 8.769313786440872|[922.593064124066...|
| 8.770732377170626|[923.833996226096...|
+------------------+--------------

In [38]:
testData.show() # it always flag featuares as input and label as output

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
| 7.443641852120096|[921.211065935867...|
| 8.283812812789947|[920.742823398151...|
| 8.450000000001149|[919.710000000012...|
| 8.620000000000013|[920.320000000003...|
| 8.947679671017102|[920.042264327172...|
| 9.130000000000672|[916.990000000007...|
| 9.780000000001193|[918.090000000011...|
|   9.8200000000003|[919.550000000001...|
|10.230000000000052|[921.370000000007...|
|10.376057996120188|[918.125062235076...|
|10.600000000000296|[920.020000000009...|
| 12.05000000000013|[923.120000000009...|
|12.054638973604598|[926.177368159633...|
|12.280000000000587|[925.290000000009...|
|12.362984078201436|[920.304856584910...|
|12.392425138555886|[920.423862879328...|
|12.480000000000315|[926.500000000001...|
| 12.52000000000047|[918.600000000006...|
|12.698545982062727|[915.914435739919...|
|12.705058755843442|[922.702458837993...|
+------------------+--------------

In [39]:
lr = LinearRegression()


In [40]:
model1 = lr.fit(trainData)


In [41]:
print("Coefficients: %s" % str(model1.coefficients))
print("Intercept: %s" % str(model1.intercept))

Coefficients: [-2.877439307825217,-1.0280416704696347,0.004921678184754102,3.055005039071246,0.048331976932163664,-3.1196841194340448,-1.1401931763202253,0.0022852807308053197]
Intercept: 2742.8186498150835


In [42]:
trainingSummary = model1.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 0
objectiveHistory: [0.0]
+-------------------+
|          residuals|
+-------------------+
|-3.1064379456547124|
|-2.8596078331150405|
|0.45709316806954803|
| -2.982479700508854|
|-1.6695325391120104|
| 1.6065376702677217|
| -4.706824307400186|
|0.42416754507627275|
|-12.483531177828644|
| -8.431435441528603|
|-5.6720128625580974|
| -9.925127170786212|
| -6.195067201787693|
| -5.110480325014823|
|-15.200455887591414|
| -9.732610992076726|
| -9.632348034871942|
|-10.287258196445412|
| -2.778159714022312|
|  4.007220463227478|
+-------------------+
only showing top 20 rows

RMSE: 14.355334
r2: 0.581634


In [43]:
predictions = model1.transform(testData)

In [44]:
predictions.show()

+------------------+--------------------+-------------------+
|             label|            features|         prediction|
+------------------+--------------------+-------------------+
| 7.443641852120096|[921.211065935867...|  16.15039688573961|
| 8.283812812789947|[920.742823398151...|  9.878446866239756|
| 8.450000000001149|[919.710000000012...|  30.74719201899643|
| 8.620000000000013|[920.320000000003...|  18.70211606610428|
| 8.947679671017102|[920.042264327172...|  9.891418351770426|
| 9.130000000000672|[916.990000000007...| 26.389427856615384|
| 9.780000000001193|[918.090000000011...| 18.300114521753585|
|   9.8200000000003|[919.550000000001...|  25.97606505418662|
|10.230000000000052|[921.370000000007...| 17.681002126726526|
|10.376057996120188|[918.125062235076...| 19.674755155055664|
|10.600000000000296|[920.020000000009...| 26.792231095288116|
| 12.05000000000013|[923.120000000009...| 27.104140028694474|
|12.054638973604598|[926.177368159633...| 1.4881934023883332|
|12.2800

In [1]:
sc.stop()

NameError: name 'sc' is not defined